In [1]:
# Read data from DataMapWales
from owslib.wms import WebMapService

# Standard Imports
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [47]:
def fetch_wms_data(wms_url, bbox, width, height, srs="EPSG:4326", format="image/png", version="1.3.0"):
    """
    Fetches map data from a WMS endpoint.

    Parameters:
        wms_url (str): The URL of the WMS endpoint.
        bbox (tuple): The bounding box (minx, miny, maxx, maxy) for the request.
        size (tuple): The width and height (respectively) of the requested image in pixels.
        srs (str): The spatial reference system (default is "EPSG:4326").
        format (str): The format of the returned image (default is "image/png").
        version (str): The WMS version to use (default is "1.3.0").

    Returns:
        bytes: The raw image data returned by the WMS service.
    """
    try:
        def get_response(layer_name):
            # Fetch the map data
            print(f"Fetching layer: {layer_name}")
            response = wms.getmap(
                layers=[layer_name],
                srs=srs,
                bbox=bbox,
                size=(width, height),
                format=format,
                transparent=True
            )
            return response.read()
        
        # Initialize the WMS service
        wms = WebMapService(wms_url, version=version)

        layers = list(wms.contents)
        
        # Check contents of the WMS service
        print("Available Layers: ", layers)
        
        if len(layers) == 0:
            print("No layers available in the WMS service.")
            return None
        elif len(layers) == 1:
            layer_name = layers[0]
            return get_response(layer_name)
        elif len(layers) > 1:
            response_list = []
            for layer in layers:
                response_list.append(get_response(layer))
            return response_list
        else: 
            print("An unexpected error occurred: No layers found.")
            return None
        
    except Exception as e:
        print(f"An error occurred while fetching WMS data: {e}")
        return None
        
def save_image(byte_data):
    """ Takes output from fetch_wms_data and saves it to a file """
    if len(byte_data) == 1:
        out = open(f'output0.png', 'wb')
        out.write(byte_data)
        out.close()
    else:
        for i, img in enumerate(byte_data):
            out = open(f'output{i}.png', 'wb')
            out.write(img)
            out.close()

In [48]:
wms_url = "https://datamap.gov.wales/capabilities/map/scheduled-monuments-and-sssi/?ows_service=wms"
output = fetch_wms_data(wms_url, bbox = (-4.5, 51.5, -3.5, 52), width = 800, height = 600)
save_image(output)

Available Layers:  ['geonode:wales_lidar_dsm_1m_hillshade_multi_cog', 'inspire-wg:Cadw_SAM', 'inspire-nrw:NRW_SSSI']
Fetching layer: geonode:wales_lidar_dsm_1m_hillshade_multi_cog
Fetching layer: inspire-wg:Cadw_SAM
Fetching layer: inspire-nrw:NRW_SSSI
